In [61]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Modules

In [94]:
from subprocess import PIPE, run, STDOUT
#from os import environ

#environ['TOMA'] = 'BBBBB'
print(run('module list', stdout=PIPE, stderr=STDOUT, shell=True).stdout.decode('utf-8'))


Currently Loaded Modules:
  1) gcc/6.3.0      3) intel-impi/2017.1.132   5) slurm/17.02.2 (S)
  2) cmake/3.10.0   4) intel-mkl/2018.1.163

  Where:
   S:  Module is Sticky, requires --force to unload or purge




# Parameters

In [11]:
from operator import mul
from functools import reduce
from datetime import datetime
from numpy import linspace

# TODO
domain = [256, 256, 256]                    
procs = [1, 1, 1]                          # 2, 4, 6, 8
execName = 'laplace3d_jacobi'               
timestamp = datetime.now()

params = {
    
    # Linear system
    'Matrix': execName.split('_')[0],
    'Domain partitions': [ 
        linspace(0, dLen, pLen + 1, endpoint=True, dtype=int).tolist() 
        for (dLen, pLen) in zip(domain, procs) ],
    'Initial guess': 'zero',
    'Exact solution': 'white-noise sine',

    # Solver
    'Iterant': execName.split('_')[1],
    'Steps': 1,
    'Max iterations': 1000000,
    'Tolerance': 1e-4,
    'Timeout [min]': 2,
    #'Truncation': 10,
    
    # Preconditioner
    #'Preconditioner Iterant': execName.split('_')[2],
    #'Preconditioner Steps': 1,
    #'Preconditioner Max iterations': 4,
    #'Preconditioner Tolerance': 0.3,
    #'Preconditioner Timeout [min]': 0.001,
    
    # Block solver
    #'Block Iterant': execName.split('_')[3],
    #'Block Steps': 1,
    #'Block Max iterations': 5,
    #'Block Tolerance': 0.1,
    #'Block Timeout [min]': 1,
    
    # Info
    'Info': 'SIAM',
    
    # Executable
    'Executable path': '~/asynchronator/build/example',
    'Executable name': execName,
    
    # Device
    'Device': 'julia',
    'Processes per node': 1,
    'Threads per process': 256,
    'Timestamp': timestamp.strftime("%Y-%m-%d %H:%M:%S"), 
    'Processes grid': procs,
    'Nodes': reduce(mul, procs),
    'Processes': reduce(mul, procs),
}

params

{'Device': 'julia',
 'Domain partitions': [[0, 256], [0, 256], [0, 256]],
 'Exact solution': 'white-noise sine',
 'Executable name': 'laplace3d_jacobi',
 'Executable path': '~/asynchronator/build/example',
 'Info': 'SIAM',
 'Initial guess': 'zero',
 'Iterant': 'jacobi',
 'Matrix': 'laplace3d',
 'Max iterations': 1000000,
 'Nodes': 1,
 'Processes': 1,
 'Processes grid': [1, 1, 1],
 'Processes per node': 1,
 'Steps': 1,
 'Threads per process': 256,
 'Timeout [min]': 2,
 'Timestamp': '2018-04-26 16:09:19',
 'Tolerance': 0.0001}

# Command

In [105]:
job = ('sbatch '
       '--job-name={executable} '
       '--nodes={nodes} '
       '--ntasks-per-node={procs} '
       '--cpus-per-task={threads} '
       '--time={timeout} '
       '--mail-user=teodor.nikolov22@gmail.com '
       '--mail-type=FAIL '
       '--error=err.txt '
       '--output=out.txt '
       '--wrap=\"srun {execDir}/{executable}\"').format(
    executable=params['Executable name'],
    execDir=params['Executable path'],
    procs=params['Processes per node'],
    nodes=params['Nodes'],
    threads=params['Threads per process'],
    timeout='{:0>2d}:{:0>2d}:00'.format(params['Timeout [min]'] // 60, 
                                        params['Timeout [min]'] % 60)
)
job

'sbatch --job-name=example.laplace3d_ajacobi --nodes=1 --ntasks-per-node=1 --cpus-per-task=256 --time=00:02:00 --mail-user=teodor.nikolov22@gmail.com --mail-type=FAIL --error=err.txt --output=out.txt --wrap="srun ~/asynchronator/build/example/example.laplace3d_ajacobi"'

# Submit

In [106]:
from subprocess import run
from pathlib import Path
from json import dump
from datetime import datetime

# Create a run directory
runDir = Path('{}_{}'.format(params['Executable name'],
                             timestamp.strftime("%Y-%m-%d_%H-%M-%S")))
runDir.mkdir(exist_ok=False)

# Write setup file
with (runDir / 'setup.json').open('w') as setupF: 
    dump(params, setupF)

# Run the job
run(job, cwd=runDir, shell=True)

CompletedProcess(args='sbatch --job-name=example.laplace3d_ajacobi --nodes=1 --ntasks-per-node=1 --cpus-per-task=256 --time=00:02:00 --mail-user=teodor.nikolov22@gmail.com --mail-type=FAIL --error=err.txt --output=out.txt --wrap="srun ~/asynchronator/build/example/example.laplace3d_ajacobi"', returncode=0)